In [2]:
import os
os.environ["PYARROW_IGNORE_EXTENSION_TYPES"] = "1"
import pandas as pd


In [3]:
pd.__version__

'2.3.1'

In [5]:
df = pd.read_parquet('../yellow_tripdata_2024-01.parquet', engine="fastparquet")

In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2.0,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186.0,79.0,2.0,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1.0,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140.0,236.0,1.0,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1.0,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236.0,79.0,1.0,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1.0,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79.0,211.0,1.0,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1.0,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211.0,148.0,1.0,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [7]:
import sklearn

In [8]:
sklearn.__version__

'1.7.0'